In [1]:
import random, gc, os, pickle, csv, time

import datasets.utils
import models.utils
from models.cls_oml_ori_v2 import OML
from models.base_models_ori import LabelAwareReplayMemory

import numpy as np

import higher
import torch
import torch.nn.functional as F
from torch.utils import data

# Constants

In [2]:
dataset_order_mapping = {
    1: [2, 0, 3, 1, 4],
    2: [3, 4, 0, 1, 2],
    3: [2, 4, 1, 3, 0],
    4: [0, 2, 1, 4, 3]
}
n_classes = 33
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
# device = torch.device('cpu')
# model_path = "/data/model_runs/original_oml/aOML-order1-2022-07-18/OML-order1-id4-2022-07-18_17-53-13.518612.pt"
# model_path = "/data/model_runs/original_oml/aOML-order1-inlr002-2022-07-31/OML-order1-id4-2022-07-31_14-53-46.456804.pt"
# model_path = "/data/model_runs/original_oml/aOML-order1-inlr005-2022-07-31/OML-order1-id4-2022-07-31_18-47-41.477968.pt"
# model_path = "/data/model_runs/original_oml/aOML-order1-inlr005-up20-2022-08-01/OML-order1-id4-2022-08-01_14-45-55.869765.pt"
# model_path = "/data/model_runs/original_oml/aOML-order1-inlr010-2022-07-31/OML-order1-id4-2022-07-31_21-18-36.241546.pt"
# model_path = "/data/model_runs/original_oml/aOML-order1-inlr020-2022-08-16/OML-order1-id4-2022-08-16_11-37-19.424113.pt"
# model_path = "/data/model_runs/original_oml/aOML-order1-inlr050-2022-08-16/OML-order1-id4-2022-08-16_14-16-12.167637.pt"

# v. SR 
# model_path = "/data/model_runs/original_oml/aOML-order1-inlr010-2022-08-29-sr/OML-order1-id4-2022-08-29_18-10-31.695669.pt"
# v. SR Query
# model_path = "/data/model_runs/original_oml/aOML-order1-inlr010-2022-08-30-sr-query/OML-order1-id4-2022-08-30_05-21-18.854228.pt"
# model_path = "/data/model_runs/original_oml/aOML-order1-inlr010-2022-14-21-sreversed/OML-order1-id4-2022-12-15_03-55-41.428520.pt"
# model_path = "/data/model_runs/original_oml/aOML-order1-inlr010-2023-01-03-srev-replay/OML-order1-id4-2023-01-03_02-59-30.904898.pt"
# model_path = "/data/model_runs/original_oml/aOML-order2-2023-01-07-srev-replay/OML-order2-id4-2023-01-07_19-01-27.774382.pt"
# model_path = "/data/model_runs/original_oml/aOML-order3-2023-01-08-srev-replay/OML-order3-id4-2023-01-08_12-03-38.782801.pt"
# model_path = "/data/model_runs/original_oml/aOML-order4-2023-01-08-srev-replay/OML-order4-id4-2023-01-08_21-01-31.303061.pt"
# model_path = "/data/model_runs/original_oml/aOML-order1-inlr010-2023-01-20-srev-replay2/OML-order1-id4-2023-01-21_19-49-04.497477.pt"
# model_path = "/data/model_runs/original_oml/aOML-order1-inlr010-2023-02-13-srev-curri/OML-order1-id4-2023-02-13_05-46-13.147873.pt"
model_path = "/data/model_runs/original_oml/aOML-order1-2023-02-18-roberta/OML-order1-id4-2023-02-18_14-23-29.188109.pt"


# memory_path = "/data/model_runs/original_oml/aOML-order1-2022-07-18/OML-order1-id4-2022-07-18_17-53-13.518639_memory.pickle"
# memory_path = "/data/model_runs/original_oml/aOML-order1-inlr002-2022-07-31/OML-order1-id4-2022-07-31_14-53-46.456828_memory.pickle"
# memory_path = "/data/model_runs/original_oml/aOML-order1-inlr005-2022-07-31/OML-order1-id4-2022-07-31_18-47-41.477992_memory.pickle"
# memory_path = "/data/model_runs/original_oml/aOML-order1-inlr005-up20-2022-08-01/OML-order1-id4-2022-08-01_14-45-55.869797_memory.pickle"
# memory_path = "/data/model_runs/original_oml/aOML-order1-inlr010-2022-07-31/OML-order1-id4-2022-07-31_21-18-36.241572_memory.pickle"
# memory_path = "/data/model_runs/original_oml/aOML-order1-inlr020-2022-08-16/OML-order1-id4-2022-08-16_11-37-19.424139_memory.pickle"
# memory_path = "/data/model_runs/original_oml/aOML-order1-inlr050-2022-08-16/OML-order1-id4-2022-08-16_14-16-12.167666_memory.pickle"
# v. SR 
# memory_path = "/data/model_runs/original_oml/aOML-order1-inlr010-2022-08-29-sr/OML-order1-id4-2022-08-29_18-10-31.695692_memory.pickle"
# v. SR Query
# memory_path = "/data/model_runs/original_oml/aOML-order1-inlr010-2022-08-30-sr-query/OML-order1-id4-2022-08-30_05-21-18.854254_memory.pickle"
# memory_path = "/data/model_runs/original_oml/aOML-order1-inlr010-2022-14-21-sreversed/OML-order1-id4-2022-12-15_03-55-41.428551_memory.pickle"
# memory_path = "/data/model_runs/original_oml/aOML-order1-inlr010-2023-01-03-srev-replay/OML-order1-id4-2023-01-03_02-59-30.904925_memory.pickle"
# memory_path = "/data/model_runs/original_oml/aOML-order2-2023-01-07-srev-replay/OML-order2-id4-2023-01-07_19-01-27.774413_memory.pickle"
# memory_path = "/data/model_runs/original_oml/aOML-order3-2023-01-08-srev-replay/OML-order3-id4-2023-01-08_12-03-38.782827_memory.pickle"
# memory_path = "/data/model_runs/original_oml/aOML-order4-2023-01-08-srev-replay/OML-order4-id4-2023-01-08_21-01-31.303088_memory.pickle"
# memory_path = "/data/model_runs/original_oml/aOML-order1-inlr010-2023-01-20-srev-replay2/OML-order1-id4-2023-01-21_19-49-04.497510_memory.pickle"
memory_path = "/data/model_runs/original_oml/aOML-order1-2023-02-18-roberta/OML-order1-id4-2023-02-18_14-23-29.188144_memory.pickle"


# new_memory_path, ext = os.path.splitext(memory_path)
# new_memory_path = new_memory_path + "_label" + ext

use_db_cache = True
cache_dir = 'tmp'

In [3]:
args = {
    "order": 1,
    "n_epochs": 1,
    "lr": 3e-5,
    "inner_lr": 0.001*5,
    "meta_lr": 3e-5,
    "model": "roberta",
    "learner": "oml",
    "mini_batch_size": 16,
    "updates": 5,
    "write_prob": 0.1,
    "max_length": 448,
    "seed": 42,
    "replay_rate": 0.01,
    "replay_every": 8000,
    "task_aware": True,
    "reverse_support": True,
    "curriculum_replay": True,
    "pln": "2fc"
}
sort_score = True
updates = args["updates"]
mini_batch_size = args["mini_batch_size"]
order = args["order"]

In [4]:
torch.manual_seed(args["seed"])
random.seed(args["seed"])
np.random.seed(args["seed"])

# Load Dataset

In [5]:
print('Loading the datasets')
test_datasets = []
for dataset_id in dataset_order_mapping[order]:
    test_dataset_file = os.path.join(cache_dir, f"{dataset_id}.cache")
    if os.path.exists(test_dataset_file):
        with open(test_dataset_file, 'rb') as f:
            test_dataset = pickle.load(f)
    else:
        test_dataset = datasets.utils.get_dataset_test("", dataset_id)
        print('Loaded {}'.format(test_dataset.__class__.__name__))
        test_dataset = datasets.utils.offset_labels(test_dataset)
        pickle.dump(test_dataset, open( test_dataset_file, "wb" ), protocol=pickle.HIGHEST_PROTOCOL)
        print(f"Pickle saved at {test_dataset_file}")
    test_datasets.append(test_dataset)
print('Finished loading all the datasets')

Loading the datasets
Finished loading all the datasets


# Load Model

In [6]:
learner = OML(device=device, n_classes=n_classes, **args)
print('Using {} as learner'.format(learner.__class__.__name__))
learner.load_model(model_path)
with open(memory_path, 'rb') as f:
#     learner.memory = pickle.load(f)
    memory_buffer = pickle.load(f)


2023-02-18 19:40:13,583 - transformers.tokenization_utils_base - INFO - loading file https://s3.amazonaws.com/models.huggingface.co/bert/roberta-base-vocab.json from cache at /root/.cache/torch/transformers/d0c5776499adc1ded22493fae699da0971c1ee4c2587111707a4d177d20257a2.ef00af9e673c7160b4d41cfda1f48c5f4cba57d5142754525572a846a1ab1b9b
2023-02-18 19:40:13,586 - transformers.tokenization_utils_base - INFO - loading file https://s3.amazonaws.com/models.huggingface.co/bert/roberta-base-merges.txt from cache at /root/.cache/torch/transformers/b35e7cd126cd4229a746b5d5c29a749e8e84438b14bcdb575950584fe33207e8.70bec105b4158ed9a1747fea67a43f5dee97855c64d62b6ec3742f4cfdb5feda
2023-02-18 19:40:14,832 - transformers.configuration_utils - INFO - loading configuration file https://s3.amazonaws.com/models.huggingface.co/bert/roberta-base-config.json from cache at /root/.cache/torch/transformers/e1a2a406b5a05063c31f4dfdee7608986ba7c6393f7f79db5e69dcd197208534.117c81977c5979de8c088352e74ec6e70f5c66096c2

Using OML as learner


In [7]:
# Setting up task dict for task-aware
memory_buffer.task_dict = {
    0: list(range(5, 9)), # AG
    1: list(range(0, 5)), # Amazon
    2: list(range(0, 5)), # Yelp
    3: list(range(9, 23)), # DBPedia
    4: list(range(23, 33)), # Yahoo
}

In [8]:
dataclass_mapper = {
    "AGNewsDataset": 0,
    "AmazonDataset": 1,
    "YelpDataset": 2,
    "DBPediaDataset": 3,
    "YahooAnswersDataset": 4
}
dataclass_mapper["AGNewsDataset"]

0

# Testing

Select specific column index per row
https://stackoverflow.com/questions/23435782/numpy-selecting-specific-column-index-per-row-by-using-a-list-of-indexes

In [9]:
def evaluate(dataloader, updates, mini_batch_size, dataname=""):
    learner.rln.eval()
    learner.pln.train()
    
    all_losses, all_predictions, all_labels, all_label_conf = [], [], [], []
    all_adaptation_time = []
    # Get Query set first. and then find supporting support set
    for query_idx, (query_text, query_labels) in enumerate(dataloader):
        print(f"Query ID {query_idx}/{len(dataloader)}")
        # The task id to optimize to for support set
        # task_idx = get_task_from_label_list(query_labels, memory_buffer.task_dict)
        task_idx = dataclass_mapper[dataname]
        
    
        support_set = []
        for _ in range(updates):
            text, labels = memory_buffer.read_batch_task(batch_size=mini_batch_size, task_idx=task_idx, sort_score=sort_score)
            support_set.append((text, labels))

        with higher.innerloop_ctx(learner.pln, learner.inner_optimizer,
                                  copy_initial_weights=False, track_higher_grads=False) as (fpln, diffopt):
            
            INNER_tic = time.time()
            # Inner loop
            task_predictions, task_labels = [], []
            support_loss = []
            for text, labels in support_set:
                labels = torch.tensor(labels).to(device)
                input_dict = learner.rln.encode_text(text)
                _repr = learner.rln(input_dict)
                output = fpln(_repr)
                loss = learner.loss_fn(output, labels)
                diffopt.step(loss)
                pred = models.utils.make_prediction(output.detach())
                support_loss.append(loss.item())
                task_predictions.extend(pred.tolist())
                task_labels.extend(labels.tolist())
            INNER_toc = time.time() - INNER_tic
            all_adaptation_time.append(INNER_toc)

            acc, prec, rec, f1 = models.utils.calculate_metrics(task_predictions, task_labels)

            print('Support set metrics: Loss = {:.4f}, accuracy = {:.4f}, precision = {:.4f}, '
                        'recall = {:.4f}, F1 score = {:.4f}'.format(np.mean(support_loss), acc, prec, rec, f1))

            # Query set is now here!
            query_labels = torch.tensor(query_labels).to(device)
            query_input_dict = learner.rln.encode_text(query_text)
            with torch.no_grad():
                query_repr = learner.rln(query_input_dict)
                query_output = fpln(query_repr) # Output has size of torch.Size([16, 33]) [BATCH, CLASSES]
                query_loss = learner.loss_fn(query_output, query_labels)
            query_loss = query_loss.item()
            # print(output.detach().size())
            # output.detach().max(-1) max on each Batch, which will return [0] max, [1] indices
            query_output_softmax = F.softmax(query_output, -1)
            query_label_conf = query_output_softmax[np.arange(len(query_output_softmax)), query_labels] # Select labels in the softmax of 33 classes

            query_pred = models.utils.make_prediction(query_output.detach())
            query_acc, query_prec, query_rec, query_f1 = models.utils.calculate_metrics(query_pred.tolist(), query_labels.tolist())
            
            print('Query set metrics: Loss = {:.4f}, accuracy = {:.4f}, precision = {:.4f}, '
                'recall = {:.4f}, F1 score = {:.4f}'.format(np.mean(query_loss), query_acc, query_prec, query_rec, query_f1))

            all_losses.append(query_loss)
            all_predictions.extend(query_pred.tolist())
            all_labels.extend(query_labels.tolist())
            all_label_conf.extend(query_label_conf.tolist())

    acc, prec, rec, f1 = models.utils.calculate_metrics(all_predictions, all_labels)
    print('Test metrics: Loss = {:.4f}, accuracy = {:.4f}, precision = {:.4f}, recall = {:.4f}, '
                'F1 score = {:.4f}'.format(np.mean(all_losses), acc, prec, rec, f1))
    return acc, prec, rec, f1, all_predictions, all_labels, all_label_conf, all_adaptation_time

In [10]:
tic = time.time()
print('----------Testing on test set starts here----------')

accuracies, precisions, recalls, f1s = [], [], [], []
all_adapt_time = []
# Data for Visualization: [data_idx, label, label_conf, pred]
data_for_visual = []

for test_dataset in test_datasets:
    print('Testing on {}'.format(test_dataset.__class__.__name__))
    test_dataloader = data.DataLoader(test_dataset, batch_size=mini_batch_size, shuffle=False,
                                      collate_fn=datasets.utils.batch_encode)
    acc, prec, rec, f1, all_pred, all_label, all_label_conf, all_adaptation_time = evaluate(dataloader=test_dataloader, updates=updates, 
                                                mini_batch_size=mini_batch_size, dataname=test_dataset.__class__.__name__)
    
    data_ids = [test_dataset.__class__.__name__ + str(i) for i in range(len(all_label))]
    data_for_visual.extend(list(zip(data_ids, all_label, all_label_conf, all_pred)))
    all_adapt_time.extend(all_adaptation_time)
#     print(data_ids)
#     print(all_label)
#     raise Exception("BREAKPOINT")
    
    accuracies.append(acc)
    precisions.append(prec)
    recalls.append(rec)
    f1s.append(f1)


print()
print("COPY PASTA - not really but ok")
for row in accuracies:
    print(row)
print()
print('Overall test metrics: Accuracy = {:.4f}, precision = {:.4f}, recall = {:.4f}, '
            'F1 score = {:.4f}'.format(np.mean(accuracies), np.mean(precisions), np.mean(recalls), np.mean(f1s)))

toc = time.time() - tic
print(f"Total Time used: {toc//60} minutes")

----------Testing on test set starts here----------
Testing on YelpDataset
Query ID 0/475
Support set metrics: Loss = 0.3582, accuracy = 0.8875, precision = 0.9095, recall = 0.8867, F1 score = 0.8837
Query set metrics: Loss = 0.5093, accuracy = 0.7500, precision = 0.7833, recall = 0.7600, F1 score = 0.7569
Query ID 1/475
Support set metrics: Loss = 0.6215, accuracy = 0.8500, precision = 0.9128, recall = 0.8567, F1 score = 0.8602
Query set metrics: Loss = 0.6629, accuracy = 0.8125, precision = 0.7667, recall = 0.8933, F1 score = 0.7911
Query ID 2/475
Support set metrics: Loss = 0.4355, accuracy = 0.9125, precision = 0.9454, recall = 0.9200, F1 score = 0.9266
Query set metrics: Loss = 0.8465, accuracy = 0.6875, precision = 0.7400, recall = 0.7167, F1 score = 0.6587
Query ID 3/475
Support set metrics: Loss = 0.3829, accuracy = 0.8750, precision = 0.9069, recall = 0.8767, F1 score = 0.8723
Query set metrics: Loss = 1.6471, accuracy = 0.5625, precision = 0.6300, recall = 0.5833, F1 score = 

Support set metrics: Loss = 0.3836, accuracy = 0.9000, precision = 0.9364, recall = 0.9033, F1 score = 0.9059
Query set metrics: Loss = 0.6814, accuracy = 0.6875, precision = 0.7250, recall = 0.7000, F1 score = 0.7000
Query ID 36/475
Support set metrics: Loss = 0.5396, accuracy = 0.8625, precision = 0.9128, recall = 0.8600, F1 score = 0.8595
Query set metrics: Loss = 0.9569, accuracy = 0.6250, precision = 0.6267, recall = 0.6667, F1 score = 0.5921
Query ID 37/475
Support set metrics: Loss = 0.3694, accuracy = 0.9000, precision = 0.9246, recall = 0.9000, F1 score = 0.9033
Query set metrics: Loss = 0.9966, accuracy = 0.5000, precision = 0.4467, recall = 0.4200, F1 score = 0.4167
Query ID 38/475
Support set metrics: Loss = 0.3928, accuracy = 0.8875, precision = 0.9167, recall = 0.8867, F1 score = 0.8884
Query set metrics: Loss = 0.6827, accuracy = 0.6875, precision = 0.5267, recall = 0.6600, F1 score = 0.5832
Query ID 39/475
Support set metrics: Loss = 0.4999, accuracy = 0.8750, precision

Support set metrics: Loss = 0.3567, accuracy = 0.8875, precision = 0.9095, recall = 0.8833, F1 score = 0.8791
Query set metrics: Loss = 0.8813, accuracy = 0.6250, precision = 0.5133, recall = 0.5333, F1 score = 0.5111
Query ID 72/475
Support set metrics: Loss = 0.5117, accuracy = 0.8250, precision = 0.8965, recall = 0.8300, F1 score = 0.8293
Query set metrics: Loss = 1.4895, accuracy = 0.5000, precision = 0.5600, recall = 0.6500, F1 score = 0.5878
Query ID 73/475
Support set metrics: Loss = 0.5739, accuracy = 0.8500, precision = 0.9128, recall = 0.8567, F1 score = 0.8642
Query set metrics: Loss = 1.1160, accuracy = 0.5000, precision = 0.5133, recall = 0.4467, F1 score = 0.4600
Query ID 74/475
Support set metrics: Loss = 0.6942, accuracy = 0.7875, precision = 0.8754, recall = 0.7900, F1 score = 0.7939
Query set metrics: Loss = 0.8438, accuracy = 0.7500, precision = 0.7667, recall = 0.7833, F1 score = 0.7614
Query ID 75/475
Support set metrics: Loss = 0.3767, accuracy = 0.9000, precision

Support set metrics: Loss = 0.5397, accuracy = 0.8625, precision = 0.9167, recall = 0.8633, F1 score = 0.8748
Query set metrics: Loss = 1.2516, accuracy = 0.4375, precision = 0.3476, recall = 0.4667, F1 score = 0.3897
Query ID 107/475
Support set metrics: Loss = 0.4526, accuracy = 0.8625, precision = 0.9250, recall = 0.8733, F1 score = 0.8786
Query set metrics: Loss = 0.8460, accuracy = 0.7500, precision = 0.8000, recall = 0.7667, F1 score = 0.7343
Query ID 108/475
Support set metrics: Loss = 0.4811, accuracy = 0.8750, precision = 0.9250, recall = 0.8800, F1 score = 0.8816
Query set metrics: Loss = 0.8357, accuracy = 0.6250, precision = 0.6200, recall = 0.7033, F1 score = 0.6143
Query ID 109/475
Support set metrics: Loss = 0.5006, accuracy = 0.8625, precision = 0.9125, recall = 0.8667, F1 score = 0.8683
Query set metrics: Loss = 1.1882, accuracy = 0.5625, precision = 0.6133, recall = 0.6133, F1 score = 0.5822
Query ID 110/475
Support set metrics: Loss = 0.5173, accuracy = 0.8375, preci

Support set metrics: Loss = 0.3517, accuracy = 0.8875, precision = 0.9128, recall = 0.8833, F1 score = 0.8768
Query set metrics: Loss = 0.8284, accuracy = 0.5625, precision = 0.5633, recall = 0.5500, F1 score = 0.5119
Query ID 142/475
Support set metrics: Loss = 0.5798, accuracy = 0.8125, precision = 0.8917, recall = 0.8167, F1 score = 0.8240
Query set metrics: Loss = 0.7896, accuracy = 0.7500, precision = 0.6476, recall = 0.7000, F1 score = 0.6597
Query ID 143/475
Support set metrics: Loss = 0.4203, accuracy = 0.8750, precision = 0.9004, recall = 0.8733, F1 score = 0.8752
Query set metrics: Loss = 0.6009, accuracy = 0.7500, precision = 0.8133, recall = 0.8100, F1 score = 0.7321
Query ID 144/475
Support set metrics: Loss = 0.5840, accuracy = 0.8625, precision = 0.9265, recall = 0.8667, F1 score = 0.8824
Query set metrics: Loss = 1.6988, accuracy = 0.5625, precision = 0.3800, recall = 0.4833, F1 score = 0.4121
Query ID 145/475
Support set metrics: Loss = 0.4903, accuracy = 0.8625, preci

...

Support set metrics: Loss = 0.5379, accuracy = 0.8000, precision = 0.8420, recall = 0.8000, F1 score = 0.8051
Query set metrics: Loss = 0.3923, accuracy = 0.8750, precision = 0.9062, recall = 0.9062, F1 score = 0.8810
Query ID 313/475
Support set metrics: Loss = 0.4754, accuracy = 0.8625, precision = 0.8830, recall = 0.8600, F1 score = 0.8601
Query set metrics: Loss = 0.5004, accuracy = 0.7500, precision = 0.8125, recall = 0.8833, F1 score = 0.8095
Query ID 314/475
Support set metrics: Loss = 0.4848, accuracy = 0.8750, precision = 0.9116, recall = 0.8800, F1 score = 0.8734
Query set metrics: Loss = 0.4929, accuracy = 0.8750, precision = 0.8214, recall = 0.8214, F1 score = 0.8163
Query ID 315/475
Support set metrics: Loss = 0.5142, accuracy = 0.8750, precision = 0.9026, recall = 0.8900, F1 score = 0.8827
Query set metrics: Loss = 0.7802, accuracy = 0.7500, precision = 0.9286, recall = 0.8095, F1 score = 0.8286
Query ID 316/475
Support set metrics: Loss = 0.4666, accuracy = 0.8875, preci

Support set metrics: Loss = 0.5039, accuracy = 0.8375, precision = 0.8932, recall = 0.8200, F1 score = 0.8350
Query set metrics: Loss = 0.5681, accuracy = 0.8750, precision = 0.8750, recall = 0.8125, F1 score = 0.8333
Query ID 348/475
Support set metrics: Loss = 0.4935, accuracy = 0.8375, precision = 0.9082, recall = 0.8200, F1 score = 0.8313
Query set metrics: Loss = 0.7194, accuracy = 0.6875, precision = 0.5833, recall = 0.6111, F1 score = 0.5630
Query ID 349/475
Support set metrics: Loss = 0.5050, accuracy = 0.8500, precision = 0.8726, recall = 0.8400, F1 score = 0.8383
Query set metrics: Loss = 0.7182, accuracy = 0.8125, precision = 0.9286, recall = 0.8810, F1 score = 0.8762
Query ID 350/475
Support set metrics: Loss = 0.5415, accuracy = 0.8000, precision = 0.8562, recall = 0.8100, F1 score = 0.7957
Query set metrics: Loss = 0.4957, accuracy = 0.8125, precision = 0.8333, recall = 0.7708, F1 score = 0.7917
Query ID 351/475
Support set metrics: Loss = 0.5382, accuracy = 0.8125, preci

Support set metrics: Loss = 0.5097, accuracy = 0.8625, precision = 0.9061, recall = 0.8500, F1 score = 0.8527
Query set metrics: Loss = 0.8053, accuracy = 0.8125, precision = 0.7619, recall = 0.7143, F1 score = 0.6905
Query ID 383/475
Support set metrics: Loss = 0.5219, accuracy = 0.8375, precision = 0.9048, recall = 0.8100, F1 score = 0.8245
Query set metrics: Loss = 0.5754, accuracy = 0.8750, precision = 0.9286, recall = 0.9238, F1 score = 0.9079
Query ID 384/475
Support set metrics: Loss = 0.4599, accuracy = 0.9000, precision = 0.9191, recall = 0.8900, F1 score = 0.8912
Query set metrics: Loss = 1.2212, accuracy = 0.6250, precision = 0.5833, recall = 0.6500, F1 score = 0.5633
Query ID 385/475
Support set metrics: Loss = 0.5848, accuracy = 0.8000, precision = 0.8524, recall = 0.7700, F1 score = 0.7815
Query set metrics: Loss = 0.9043, accuracy = 0.6250, precision = 0.5833, recall = 0.6354, F1 score = 0.5821
Query ID 386/475
Support set metrics: Loss = 0.5056, accuracy = 0.8500, preci

Support set metrics: Loss = 0.4716, accuracy = 0.8250, precision = 0.8940, recall = 0.8000, F1 score = 0.8163
Query set metrics: Loss = 0.6636, accuracy = 0.7500, precision = 0.8000, recall = 0.8000, F1 score = 0.7667
Query ID 418/475
Support set metrics: Loss = 0.5334, accuracy = 0.8625, precision = 0.9071, recall = 0.8700, F1 score = 0.8685
Query set metrics: Loss = 0.6767, accuracy = 0.8125, precision = 0.8125, recall = 0.7708, F1 score = 0.7667
Query ID 419/475
Support set metrics: Loss = 0.5413, accuracy = 0.8250, precision = 0.8818, recall = 0.8000, F1 score = 0.7961
Query set metrics: Loss = 0.8991, accuracy = 0.6250, precision = 0.7778, recall = 0.6667, F1 score = 0.6786
Query ID 420/475
Support set metrics: Loss = 0.4817, accuracy = 0.8875, precision = 0.9226, recall = 0.8800, F1 score = 0.8848
Query set metrics: Loss = 0.8914, accuracy = 0.8125, precision = 0.6852, recall = 0.7778, F1 score = 0.7185
Query ID 421/475
Support set metrics: Loss = 0.4961, accuracy = 0.9000, preci

Support set metrics: Loss = 0.5764, accuracy = 0.7875, precision = 0.8360, recall = 0.7700, F1 score = 0.7602
Query set metrics: Loss = 0.1567, accuracy = 1.0000, precision = 1.0000, recall = 1.0000, F1 score = 1.0000
Query ID 453/475
Support set metrics: Loss = 0.4293, accuracy = 0.8750, precision = 0.9240, recall = 0.8900, F1 score = 0.8904
Query set metrics: Loss = 0.5192, accuracy = 0.8750, precision = 0.9375, recall = 0.8958, F1 score = 0.8917
Query ID 454/475
Support set metrics: Loss = 0.5381, accuracy = 0.8000, precision = 0.8546, recall = 0.7800, F1 score = 0.7730
Query set metrics: Loss = 0.3337, accuracy = 0.8750, precision = 0.8000, recall = 0.8500, F1 score = 0.8167
Query ID 455/475
Support set metrics: Loss = 0.5461, accuracy = 0.8125, precision = 0.8551, recall = 0.8100, F1 score = 0.8049
Query set metrics: Loss = 0.4642, accuracy = 0.8750, precision = 0.8056, recall = 0.8519, F1 score = 0.8138
Query ID 456/475
Support set metrics: Loss = 0.5596, accuracy = 0.8750, preci

In [11]:
_model_path0 = os.path.splitext(model_path)[0]
csv_filename = _model_path0 + "_update"+ str(updates) +"_results.csv" # for selective replay
with open(csv_filename, 'w') as csv_file:
    csv_writer = csv.writer(csv_file)
    csv_writer.writerow(["data_idx", "label", "label_conf", "pred"])
    csv_writer.writerows(data_for_visual)
print(f"Done writing CSV File at {csv_filename}")

Done writing CSV File at /data/model_runs/original_oml/aOML-order1-2023-02-18-roberta/OML-order1-id4-2023-02-18_14-23-29.188109_update5_results.csv


In [12]:
# Log Time for Inference
_model_path0 = os.path.splitext(model_path)[0]
time_txt_filename = _model_path0 + "_update"+ str(updates) +"_time_inference.csv" 
with open(time_txt_filename, 'w') as csv_file:
    csv_writer = csv.writer(csv_file)
    csv_writer.writerow(["time_id", "time"])
    csv_writer.writerow(["Total Time", f"{toc//60} minutes"])
    csv_writer.writerow(["mean Adapt Time", f"{np.mean(all_adapt_time)} s"])
print(f"Done writing Time CSV File at {time_txt_filename}")

Done writing Time CSV File at /data/model_runs/original_oml/aOML-order1-2023-02-18-roberta/OML-order1-id4-2023-02-18_14-23-29.188109_update5_time_inference.csv
